In [ ]:
%%capture
import os
import re

if "COLAB_" not in "".join(os.environ.keys()):
	!pip install unsloth
else:
	# Do this only in Colab notebooks!
	import torch

	v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
	xformers = "xformers==" + ("0.0.33.post1" if v == "2.9" else "0.0.32.post2" if v == "2.8" else "0.0.29.post3")
	!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
	!pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
	!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048  # Choose any! Unsloth auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
	model_name="unsloth/Qwen2.5-Coder-14B-Instruct",
	max_seq_length=max_seq_length,
	dtype=dtype,
	load_in_4bit=load_in_4bit,
	# token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
	model,
	r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
	target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
					"gate_proj", "up_proj", "down_proj", ],
	lora_alpha=16,
	lora_dropout=0,  # Supports any, but = 0 is optimized by Unsloth
	bias="none",  # Supports any, but = "none" is optimized by Unsloth
	# [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
	use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
	random_state=3407,
	use_rslora=False,  # Unsloth supports rank stabilized LoRA
	loftq_config=None,  # And LoftQ
)